# Lesson 02 - CRAM Basics

CRAM stands for Cognitive Robot Abstract Machine. Today we are working with the CRAM plan-executive, which is a framework for writing high-level procedures for robots. High-level, meaning, the procedures in this framework are agnostic of the underlying platform. They can be executed on different robots (PR2, TiaGo, Pepper, HSR, Boxy, Donbot), in different simulations (Bullet, Unreal, MuJoCo), with different motion planners (Giskard, MoveIt, MoveBase, NaiveKinematics), knowledge representations (KnowRob, RALF, internal heuristics), perception systems (Robosherlock, RoboCudo, sim-based perception)... All of these can be easily exchanged by switching out the process-modules for a specific platform, which is like plugging in a different pair of headphones or connect speakers instead, and still hear the same music. The high-level plan stays the same, the music stays the same, but the target output changes. Our platform today is Bullet Physics, a lightweight simulator, that's tightly coupled into CRAMs geometric reasoning.


## Compiling the CRAM code

Steel Bank Common Lisp (SBCL) is the interpreter for our code. To use CRAM, we first need to compile the code to make it available.`asdf:load-system` loads an ASDF (Another System Definition Facility) system into the SBCL kernel. You will remember this line from lesson 0.

In [ ]:
(asdf:load-system :cram-pr2-pick-place-demo)

Every time you reboot Docker or restart the kernel, even when you switch to a different lesson, you need to compile the package first. Every lesson has its own SBCL kernel.

When the pick & place package is loaded, it registers lots of initialization functions. The following will boot up the current configuration of initialization. It can also be used to reboot the whole simluation system.

In [ ]:
;; Warning: resets the core processes of the simulation
(roslisp-utilities:startup-ros)

If you don't want to reboot all the CRAM processes with `startup-ros` but only clear the Bullet World of its objects, use this

In [ ]:
;; Warning: resets simulation and deletes objects
(demo::init-projection)

When the simulation hangs up, which happens sometimes, this may bring it back to life. If it doesn't, hit that *Stop* button at the top to kill leftover processes and try again. Eventually you may need to restart the Docker process.

In [ ]:
;; Warning: respawning the window may result in a kernel crash. Restart the lecture via docker to recover from that.
(btr-utils:reset-debug-window)

## The Tutorial Package

The following block defines a new package, called *cram-bullet-world-tutorial*. This is the package we're going to write our code in.

In [ ]:
(defpackage cram-bullet-world-tutorial
  (:nicknames :btw-tut)                         ;; an alias for the package
  (:use :common-lisp :cram-prolog :desig :exe)) ;; inherit other packages' namespace to make our code better readable

As you remember from the previous lesson, packages are like namespaces. Our package has a nickname, which we can use to switch into the package.

In [ ]:
(in-package :btw-tut)

There is the function `describe` that gives explanation about symbols and how they are bound. Let's define a global parameter to check this out.

In [ ]:
(defparameter *descriptive-parameter-name* "Lisp rules")

In [ ]:
(describe '*descriptive-parameter-name*)

In [ ]:
(describe '*package*)

## Poses in 3D space

In 3D space we can define a point with x, y and z. We use the `cl-transforms` package to do so.

In [ ]:
(cl-transforms:make-3d-vector 1.0 2.0 3.0)

A point is just a point. Every object is at a certain position in the room, but we don't know how it is oriented. Orientations are commonly represented as a rotation matrix, rotation vector, euler angles or quaternion. CRAM works with quaternions, which are very hard to read for humans, but are very nice to work with. Like rotation matrices, quaternions can the concatenated to calculate a chain of relative orientations.

In [ ]:
(cl-transforms:make-identity-rotation)

See that a quaternion is composed of four values: ax, ay, az, and w. Since quaternions are so hard construct by hand, I usually translate them from euler angles.

In [ ]:
(cl-transforms:euler->quaternion)                 ;; is the identity rotation
(cl-transforms:euler->quaternion :az pi)          ;; rotates 180 degree around the z-axis. The last value is float-imprecision.
(cl-transforms:euler->quaternion :az (* pi 0.5))  ;; rotates 90 degree to the left
(cl-transforms:euler->quaternion :az (* pi -0.5)) ;; rotates 90 degree to the right

(describe 'cl-transforms:euler->quaternion)

The difference between a position and a pose is, that a position only describes a point, while a pose is the combination of position and orientation.

In [ ]:
(cl-transforms:make-pose (cl-transforms:make-3d-vector 1.0 2.0 3.0)
                         (cl-transforms:make-identity-rotation))

What's missing is a reference to something in the room. Every pose should have a reference frame, something that already has a fixed pose in the world, such that we can define a pose w.r.t. that other pose in the world. In our case this origin frame is called `map`. Such a relative pose also gets a `stamp` (like in time-stamp), but we can ignore that for now.

In [ ]:
(cl-transforms-stamped:make-pose-stamped "map" 0.0
                                         (cl-transforms:make-3d-vector 1.0 2.0 3.0)
                                         (cl-transforms:make-identity-rotation))

To make our code small, let's define a function that constructs a pose for us

In [ ]:
(defun make-pose (frame point euler)
    (declare (type string frame)      ;; verifies the type of 'frame' as string
             (type list point euler)) ;; and list for 'point' and 'euler'
    ;; this is the documentation string
    "Creates a cl-transforms pose in 3D space w.r.t. the given frame, point and euler.
    frame - a string of the reference frame in the world.
    point - given as a list of x y z coordinates, 
    euler - given as rotations around the x, y and z axis. (right-hand-rule)"
    ;; this checks if the arguments 'point' and 'euler' both have length 3
    (if (and (eq (length point) 3) (eq (length euler) 3))
        ;; destructuring-bind can assign the contents of a list to local variables.
        ;; here it binds the 3 values of the 'euler' list to ax, ay and az
        ;; mapcar with eval is required here to evaluate formulae like (* pi 0.5)
        (destructuring-bind (ax ay az) (mapcar #'eval euler)
          (cl-transforms-stamped:make-pose-stamped frame 0.0 
                                                   (apply #'cl-transforms:make-3d-vector point)
                                                   (cl-transforms:euler->quaternion :ax ax :ay ay :az az)))
        ;; else-branch
        (format T "point ~a or euler ~a is not of length 3" point euler)))

In [ ]:
(make-pose "map" '(1.0 2.0 3.0) '(0.0 0.0 (* pi 0.5)))

## Visualizing

All these poses seem pretty abstract, so let's visualize them in Bullet. The function `btr:add-vis-axis-object` can be applied to poses, physical objects and robot parts.

In [ ]:
;; try and mess around with the poses values
(btr:add-vis-axis-object (make-pose "map" '(0.0 0.0 2.0) '((* pi 0.0) (* pi 0.0) (* pi 0.0))))

These are some examples to visualize the pose of part of the environment and the robot.

In [ ]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-environment-object) "sink_area_left_middle_drawer_handle"))

In [ ]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "l_gripper_tool_frame"))

In [ ]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "base_laser_link"))

In [ ]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "torso_lift_link"))

In [ ]:
(btr:add-vis-axis-object (btr:link-pose (btr:get-robot-object) "wide_stereo_optical_frame"))

To get a full list of the environment and robot parts, this will retreive all possible frames. We use Alexandria here to extract the keys of the hash-table, in which the links of the environment are defined.

In [ ]:
(alexandria:hash-table-keys (slot-value (slot-value (btr:get-environment-object) 'btr:urdf) 'cl-urdf:links))

In [ ]:
(alexandria:hash-table-keys (slot-value (slot-value (btr:get-robot-object) 'btr:urdf) 'cl-urdf:links))

We can also visualize areas. The following calculates and shows the distribution of poses that are suitable for the PR2 to stand, in order to reach the sink-area-left-middle-drawer-handle of the kitchen.

In [ ]:
;; this takes a while to compute. lay back and observe.
(btr-utils:visualize-designator-costmaps
 (a location
    (reachable-for pr2)
    (arm left)
    (object (an object
                (type cupboard)
                (urdf-name sink-area-left-middle-drawer-handle)
                (part-of iai-kitchen)))))

## Designators
That previous code block seems wierd. It looks completely different from the Lisp code that we previously saw. 

## Moving Around



In [ ]:
(defun make-pose (reference-frame pose)
    "Creates a cl-transforms pose in 3D space w.r.t. the given `reference-frame' and `pose'.
    The pose is given as a list of two lists, where the first are the x y z coordinates, 
    the second is the quaternion of qx qy qz w, like this: '((2.0 0.0 1.5) (0.0 0.0 0.0 1.0))"
  (cl-transforms-stamped:make-pose-stamped
   reference-frame 0.0
   (apply #'cl-transforms:make-3d-vector (first pose))
   (apply #'cl-transforms:make-quaternion (second pose))))

In [ ]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal (make-pose "map" '((0 1 0) (0 0 0 1)))))
    (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal)))))))

In [ ]:
(defparameter *base-pose-near-table*
  (make-pose "map" 
             '((-2.2d0 -0.20d0 0.0d0) 
               (0.0d0 0.0d0 -0.707d0 0.707d0))))

In [ ]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal *base-pose-near-table*))
     (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal)))))))

Advanced definition of poses with euler angles provides a quaternion.

In [ ]:
(cl-transforms:euler->quaternion :az (* pi 0.5)) ;; rotation of pi/2 around the z-axis = 90 degree to the left

The resulting quaternion can be used to specify rotations around an axis. Here we a creating a pose without the help of the `make-pose` function. Fiddle arbound with the `-0.50` factor to PI, and move the robot again, to see the resulting rotation.

In [ ]:
(defparameter *base-pose-near-table*
  (cl-transforms-stamped:make-pose-stamped 
   "map" 0.0 
   (cl-transforms:make-3d-vector -2.2d0 -0.20d0 0.0d0)
   (cl-transforms:euler->quaternion :az (* pi -0.50))))

## Spawning Objects

In [ ]:
(defun spawn-object (spawn-pose &optional (obj-type :bottle) (obj-name 'bottle-1) (obj-color '(1 0 0)))
    "Spawns an object into simulation.
    spawn-pose: list - pose coordinates, like this: '((x y z) (qx qy qz w))
    optional:
      obj-type: keyword - the type of object to spawn, default = :bottle (case-insensitive)
      obj-name: symbol - name of the object, default = 'bottle-1 (case-insensitive)
      obj-color: list - (r g b) color value for the object with range 0.0-1.0, default is red = '(1 0 0)"
  (unless (assoc obj-type btr::*mesh-files*)
    (btr:add-objects-to-mesh-list "cram_pr2_pick_place_demo"))
  (btr-utils:spawn-object obj-name obj-type :color obj-color :pose spawn-pose)
  (btr:simulate btr:*current-bullet-world* 10))

To find all possible `obj-type`, the parameter `btr::*mesh-files*` contains the mesh files as key-value pairs between keyword and the location of their mesh-file. We can map over that list and only return their keys. 

In [ ]:
;; the parameter btr::*mesh-files* contains the mesh files 
(map 'list #'first btr::*mesh-files*)

Spawn bottle

In [ ]:
(spawn-object '((-2.323 -1 0.82) (0 0 0 1)))

In [ ]:
(spawn-object '((1.44 1.28 0.85) (0 0 0 1)) :cube 'cube-1)

In [ ]:
(spawn-object '((0 0 0.1) (0 0 0 1)) :weisswurst 'wurst-1 '(0 1 0))

In [ ]:
(demo::initialize)

## Intermezzo: About Designators

Action Designators: Going is an Action like searching, looking, fetching, delivering etc. 

Object Designators

Location Designators

Motion Designators

## Perceiving Objects

In [ ]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal *base-pose-near-table*))
    (cpl:par
      ;; Moving the robot near the table.
      (perform (an action
                   (type going)
                   (target (a location
                              (pose ?navigation-goal)))))
      ;; Increasing the height of the torso by setting the joint angle to 0.3 meters
      (perform (a motion
                  (type moving-torso)
                  (joint-angle 0.3)))
      (perform (an action
                  (type positioning-arm)
                  (left-configuration park)
                  (right-configuration park))))))

In [ ]:
(defparameter *downward-look-coordinate*
 (make-pose "base_footprint" '((0.65335d0 0.076d0 0.758d0) (0 0 0 1))))
;; This coordinate frame has base_footprint as reference, which is the reference
;; frame of PR2's base.

In [ ]:
(urdf-proj:with-simulated-robot
  (let ((?looking-direction *downward-look-coordinate*))
    (perform (an action
                 (type looking)
                 (target (a location 
                            (pose ?looking-direction)))))))

In [ ]:
(urdf-proj:with-simulated-robot
  (perform (an action
               (type detecting)
               (object (an object
                          (type bottle))))))

In [ ]:
(defparameter *perceived-bottle* nil)
(urdf-proj:with-simulated-robot
    (setf *perceived-bottle* 
          (perform (an action
                       (type detecting)
                       (object (an object
                                   (type bottle)))))))

## Picking up objects

In [ ]:
(urdf-proj:with-simulated-robot 
  (let ((?perceived-bottle *perceived-bottle*))
   (perform (an action
                (type picking-up)
                 (arm (right))
                 (grasp right-side)
                 (object ?perceived-bottle)))
  ;; Parking the right arm after grasping, bringing the bottle close to the robot
  ;; and freeing up the field of view.
    (exe:perform
           (desig:an action
                     (type parking-arms)
                     (arms (right))))))

## Placing Objects

In [ ]:
(defparameter *final-object-destination*
  (make-pose "map" '((-0.8 2 0.9) (0 0 0 1))))
 
(defparameter *base-pose-near-counter*
  (make-pose "map" '((-0.150d0 2.0d0 0.0d0) (0.0d0 0.0d0 -1.0d0 0.0d0))))

In [ ]:
(roslisp-utilities:startup-ros)

In [ ]:
(demo::initialize)

(spawn-object '((-2.323 -1 0.82) (0 0 0 1)))

(urdf-proj:with-simulated-robot
 (let ((?navigation-goal 
        (cl-transforms-stamped:make-pose-stamped 
            "map" 0.0 
         (cl-transforms:make-3d-vector -2.2d0 -0.20d0 0.0d0)
         (cl-transforms:euler->quaternion :az (* pi -0.50)))))
      (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal))))))
 

  (let ((?looking-direction (make-pose "base_footprint" '((0.65335d0 0.076d0 0.758d0) (0 0 0 1)))))
    (perform (an action
                 (type looking)
                 (target (a location 
                            (pose ?looking-direction))))))
 
 (let ((?perceived-bottle 
        (perform (an action
                     (type detecting)
                     (object (an object
                                 (type bottle))))))
       (?nav-goal (make-pose "map" '((-0.15d0 2.0d0 0.0d0) (0.0d0 0.0d0 -1.0d0 0.0d0))))
       (?drop-pose (make-pose "map" '((-0.8 2 0.9) (0 0 0 1)))))
  
  (perform (an action
                (type picking-up)
                 (arm (right))
                 ;(side right-side)
                 (object ?perceived-bottle)))
  ;; Parking the right arm after grasping, bringing the bottle close to the robot
  ;; and freeing up the field of view.
    (perform (an action
                 (type parking-arms)))
   (perform (a motion 
               (type moving-torso)
               (joint-angle 0.3)))
   (perform (an action
                (type going)
                (target (a location
                           (pose ?nav-goal)))))
 
   (perform (an action
                (type placing)
                (arm (right))
                (object ?perceived-bottle)
                (target (a location
                           (pose ?drop-pose)))))))

In [ ]:


(defun move-kitchen-joint (&key (joint-name "iai_fridge_door_joint")
                             (joint-angle 0.2d0))
  (btr:set-robot-state-from-joints
   `((,joint-name  ,joint-angle))
   (btr:get-environment-object)))

(defun add-objects-to-mesh-list (&optional (ros-package "cram_bullet_world_tutorial"))
  (mapcar (lambda (object-filename-and-object-extension)
            (declare (type list object-filename-and-object-extension))
            (destructuring-bind (object-filename object-extension)
                object-filename-and-object-extension
              (let ((lisp-name (roslisp-utilities:lispify-ros-underscore-name
                                object-filename :keyword)))
                (pushnew (list lisp-name
                               (format nil "package://~a/resource/~a.~a"
                                       ros-package object-filename object-extension)
                               nil)
                         btr::*mesh-files*
                         :key #'car)
                lisp-name)))
          (mapcar (lambda (pathname)
                    (list (pathname-name pathname) (pathname-type pathname)))
                  (directory (physics-utils:parse-uri
                              (format nil "package://~a/resource/*.*" ros-package))))))
                              

;;;; **** ARM Parking
(defun park-arms ()
  (pp-plans::park-arms))

(defun park-arm (arm)
  (pp-plans::park-arms :arm arm))

;;**** Failure Handler
(defmacro handle-failure (errors program-body &body error-handler-body)
  `(progn
    (cpl:with-failure-handling
        ((,errors (e)
           (print e)
           ,@error-handler-body))
    ,@program-body)))


(defparameter *pose-bottle-1*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -2 -0.9d0 0.86d0)
   (cl-transforms:make-identity-rotation)))

(defparameter *pose-bottle-2*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -0.8 2 0.9)
   (cl-transforms:make-identity-rotation)))

(defparameter *pose-meal-table*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -0.15 2.0 0)
   (cl-transforms:make-quaternion 0.0d0 0.0d0 -1.0d0 0.0d0)))

(defparameter *pose-counter*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -1.8547d0 -0.381d0 0.0d0)
   (cl-transforms:axis-angle->quaternion (cl-transforms:make-3d-vector 0 0 1) (/ pi -2))))

(defun spawn-two-bottles ()
  (unless (assoc :bottle btr::*mesh-files*)
    (add-objects-to-mesh-list))
  (prolog:prolog
   `(and (btr:bullet-world ?world)
         (assert (btr:object ?world :mesh bottle-1 ((-2 -0.9 0.860) (0 0 0 1))
                             :mass 0.2 :color (1 0 0) :mesh :bottle))
         (assert (btr:object ?world :mesh bottle-2 ((-0.8 2 0.9) (0 0 0 1))
                             :mass 0.2 :color (0 1 0) :mesh :bottle))
         (btr:simulate ?world 100))))

(defun navigate-to (?navigation-goal)
  (exe:perform (desig:a motion
                        (type going)
                        (pose ?navigation-goal))))

(defun look-at (?point-of-interest)
  (exe:perform (desig:a motion
                        (type looking)
                        (pose ?point-of-interest))))

(defun get-perceived-bottle-desig ()
  (let* ((?bottle-desig (desig:an object (type bottle)))
         (?perceived-bottle-desig (exe:perform
                                   (desig:a motion
                                            (type detecting)
                                            (object ?bottle-desig)))))
    ?perceived-bottle-desig))

(defun pick-up (?object-designator &optional (?arm :right))
  (exe:perform (desig:an action
                         (type picking-up)
                         (arm ?arm)
                         (object ?object-designator))))

(defun place-down (?pose ?object ?arm)
  (exe:perform (desig:an action
                         (type placing)
                         (arm ?arm)
                         (object ?object)
                         (target (desig:a location (pose ?pose))))))

(defun test-switch-two-bottles ()
  (spawn-two-bottles)
  (proj:with-projection-environment urdf-proj:urdf-bullet-projection-environment
    (cpl:top-level
      ;; Go to counter top and perceive bottle
      (let ((?navigation-goal *pose-counter*)
            (?ptu-goal 
              (cl-transforms-stamped:make-pose-stamped
               "base_footprint"
               0.0
               (cl-transforms:make-3d-vector 0.65335d0 0.076d0 0.758d0)
               (cl-transforms:make-identity-rotation))))
        (cpl:par
          ;; Move torso up
          (exe:perform
           (desig:a motion (type moving-torso) (joint-angle 0.3)))
          (exe:perform
           (desig:an action
                     (type parking-arms)))
          (navigate-to ?navigation-goal))
        (look-at ?ptu-goal))
      ;; Pick up bottle-1 with right arm.
      (let ((?perceived-bottle-1 (get-perceived-bottle-desig)))
        (pick-up ?perceived-bottle-1 :right)
        (exe:perform
         (desig:an action
                   (type parking-arms)
                   (arms (:right))))
        ;; Move to the meal table
        (let ((?pose *pose-meal-table*))
          (navigate-to ?pose))
        ;; Pick up bottle-2 with left arm
        (let ((?perceived-bottle-2 (get-perceived-bottle-desig)))
          (pick-up ?perceived-bottle-2 :left)
          ;; Move left arm out of sight
          (exe:perform
           (desig:an action
                     (type parking-arms)
                     (arms (:left))))
          ;; Place bottle-1 on second table
          (let ((?drop-pose *pose-bottle-2*))
            (place-down ?drop-pose ?perceived-bottle-1 :right))
          ;; Move right arm out of sight
          (exe:perform
           (desig:an action
                     (type parking-arms)
                     (arms (:right))))
          ;; Move to the counter table
          (let ((?navigation-goal *pose-counter*))
            (navigate-to ?navigation-goal))
          ;; Place bottle-2 on the counter
          (let ((?drop-pose *pose-bottle-1*))
            (place-down ?drop-pose ?perceived-bottle-2 :left))
          (exe:perform
           (desig:an action
                     (type parking-arms))))))))